In [ ]:
#| export
import picos
import numpy as np
from itertools import combinations
from collections.abc import Iterable
from bounce.utils import state2str, simplify_layout

In [ ]:
#| default_exp sdp

In [ ]:
#| include: false
from nbdev.showdoc import show_doc
%load_ext autoreload
%autoreload 2

In [ ]:
#| export
def picos2np(variable):
    "Converts picos variable (even sparse) to numpy matrix."
    return picos.expressions.data.cvx2np(variable.value)

## SdP formalism

In order to formulate and solve the SdP there are two main needed items: 
* an object containing the problem details in terms of `picos.Constant` with a `to_sdp()` method, such as a `Hamiltonian` with `Hamiltonian.to_sdp()`.
* a layout based in `np.ndarray`s, e.g., `L = [np.array([0, 1]), np.array([0, 1, 2])` determining the problem constraints.

In [ ]:
#| export
class SdPSolver:
    def __init__(self, solver='cvxopt'): self.solver = solver
        
    def solve(self):
        raise NotImplementedError("Please define an appropiate `solve` method" + 
                                  " for your problem in the `SdPSolver`.")
        
    def ojimetro(self):
        raise NotImplementedError("Please define a cost estimation function method" + 
                                  " suitable for your problem in the `SdPSolver`.")
        

The `SdPSolver` must be tailored to the problem instances that we aim to solve. Defining the problem and the solver should go hand-to-hand and they are handled by the `SdPEnvironment` during the optimization. Essentially, the environment takes the result of `problem.to_sdp()` and feeds it into the solver.  

`SdPSolver`s have two main methods: `solve` and `ojimetro`. The first one, solves a problem instance given a layout, such as `solver.solve(problem.to_sdp(), layout)`. The second one estimates the cost associated to solving the SdP given a layout, with the syntax `solver.ojimetro(layout)`. We can choose any metric for the cost, for example, the amount of free variables in the resulting SdP. 

## Ground state energy approximation

Finding the ground state energy of a quantum Hamiltonian is a fundamental problem in quantum physics. It constitutes global optimization problem over all possible quantum states $\rho$ of the form $$E_0=\min_\rho \text{Tr}\left[\rho H\right]\,,$$ where $E_0$ denotes the ground state energy for the Hamiltonian $H$. Nonetheless, the complexity of such optimization task grows exponentially with the physical system's size, so we often rely on approximations. Here, we consider a relaxation of the problem based on an SdP.

In [ ]:
#| export
class SdPEnergySolver(SdPSolver):
    """Solver to compute lower bounds of the ground state energy of local Hamiltonians. It
        follows the method described in [1] (see refs below) to formulate the objective and
        enforce the problem constraints."""
        
    def solve(self, interactions, layout):
        """Creates and solves the SdP associated to the given layout and Hamiltonian interactions.
        The result is a lower bound of the Hamiltonian's ground state energy."""
        layout = simplify_layout(layout)
        problem = picos.Problem(solver=self.solver)
        variables = [(sites, picos.HermitianVariable('rho'+','.join(map(str, sites)), 
                                                    (2**len(sites), 2**len(sites)))) for sites in layout]
        objective = self._build_energy_objective(variables, interactions)
        
        compatibility_constraints = self._get_compatibility_constraints(variables)
        problem.add_list_of_constraints([rho >> 0 for _, rho in variables])
        problem.add_list_of_constraints([picos.trace(rho) == 1 for _, rho in variables])
        problem.add_list_of_constraints(compatibility_constraints)
        problem.set_objective('min', objective)
        
        try:    
            problem.solve()
            result = np.real(objective.value)
        except: 
            result = None
        return result
    
    @staticmethod
    def ojimetro(layout):
        "Estimates the amount of free parameters involved in the SdP associated to the `layout`."
        layout = simplify_layout(layout)
        all_variables = np.sum([2**(2*len(sites)) for sites in layout])
        intersections = []
        for k, sites1 in enumerate(layout[:-1]):
            for sites2 in layout[k+1:]:
                intersections.append(np.intersect1d(sites1, sites2))
        intersections = simplify_layout(intersections)
        dep_variables = np.sum([2**(2*len(sites)) for sites in intersections])
        return all_variables - dep_variables   
        
    def _build_energy_objective(self, variables, interactions):
        "Builds the optimization objective: the expected energy."
        objective = 0
        for support, h in interactions:
            supported = False
            for sites, rho in variables:
                common, _, idx = np.intersect1d(support, sites, return_indices=True)
                if len(common) == len(support): 
                    rdm = rho.partial_trace(self._complementary_system(idx, len(sites)))
                    objective += (rdm | h) # Tr(rdm·H')
                    supported = True
                    break
            if not supported:
                eigenvalues, _ = np.linalg.eigh(picos2np(h))
                objective += min(eigenvalues)
        return objective
    
    def _get_compatibility_constraints(self, variables):
        "Instantiates the compatibility constraints between the reduced density matrices."
        compatibility_constraints = []
        for k1, (sites1, rho1) in enumerate(variables):
            for k2 in range(k1+1, len(variables)):
                sites2, rho2 = variables[k2]
                common, idx1, idx2 = np.intersect1d(sites1, sites2, return_indices=True)
                if len(common) > 0:
                    partial_trace1 = rho1.partial_trace(self._complementary_system(idx1, len(sites1)))
                    partial_trace2 = rho2.partial_trace(self._complementary_system(idx2, len(sites2)))
                    constraint = partial_trace1 - partial_trace2 == 0
                    compatibility_constraints.append(constraint)
        return compatibility_constraints
    
    @staticmethod
    def _complementary_system(subsystem, size):
        "Obtains the complementary system of subsystem."
        return list(map(int, np.setdiff1d(np.arange(size), subsystem)))

The `SdPEnergySolver` solves the problem of finding lower bounds to the ground state energy of local Hamiltonians. Hence, the `SdPEnvironment` should receive a `Hamiltonian` as problem instance together with this solver. We use this case to illustrate the general behaviour of the `SdPSolvers` and how to use them.

In [ ]:
#| include: false
from bounce.hamiltonian import XXHamiltonian, Chain1D
from bounce.utils import fill_layout

In [ ]:
N = 6
chain = Chain1D(N)
B, J = 1., [i%3 for i in range(N)]
H = XXHamiltonian(chain, B, J)

### Solving a problem instance

Now that we have defined a problem, we use the `solve` method to define the objective function, which corresponds to the expected energy, and the problem constraints, given by a layout. 

In [ ]:
show_doc(SdPEnergySolver.solve)

<h4 id="SdPEnergySolver.solve" class="doc_header"><code>SdPEnergySolver.solve</code><a href="__main__.py#L7" class="source_link" style="float:right">[source]</a></h4>

> <code>SdPEnergySolver.solve</code>(**`interactions`**, **`layout`**)

Creates and solves the SdP associated to the given layout and Hamiltonian interactions.
The result is a lower bound of the Hamiltonian's ground state energy.

The method takes the `Hamiltonian.interactions`, provided by the `Hamiltonian.to_sdp()` method, and a layout with which we define the set of compatibility constraints imposed in the SdP, as we explain in [[1]](https://arxiv.org/abs/2103.03830). 

Given the Hamiltonian, let's define a layout in order to compute an energy bound.

In [ ]:
simple_layout = fill_layout([np.array([0, N-1])], N)
simple_layout

[array([0, 5]), array([1]), array([2]), array([3]), array([4])]

And now we can solve the associated SdP to the Hamiltonian with the given set of constraints.

In [ ]:
solver = SdPEnergySolver()
solver.solve(H.to_sdp(), simple_layout)

-15.999999999012134

Tightening the constraints, we can obtain a better energy bound. For instance, adding a 3-body element to the previous set we will obtain a higher energy.

In [ ]:
stronger_layout = fill_layout([np.array([1, 2, 3]), np.array([0, N-1])], N)
stronger_layout

[array([1, 2, 3]), array([0, 5]), array([4])]

In [ ]:
solver.solve(H.to_sdp(), stronger_layout)

-12.472135949820899

### Cost estimation

A key element of any optimization problem is its computational cost. We can take any consistent measure of the problem's cost as a valid estimation for the posterior optimization with the reinforcement learning framework. For example, since the actual complexity of solving an SdP depends on the algorithm that we use, we take the number of free parameters as a metric for the associated cost.  

In order to get a cost estimation, we rely on the `ojimetro` method.

In [ ]:
show_doc(SdPEnergySolver.ojimetro)

<h4 id="SdPEnergySolver.ojimetro" class="doc_header"><code>SdPEnergySolver.ojimetro</code><a href="__main__.py#L29" class="source_link" style="float:right">[source]</a></h4>

> <code>SdPEnergySolver.ojimetro</code>(**`layout`**)

Estimates the amount of free parameters involved in the SdP associated to the `layout`.

Following our example, the first SdP was a wild relaxation of the original problem.

In [ ]:
solver.ojimetro(simple_layout)

31

However, the second one was a tighter one.

In [ ]:
solver.ojimetro(stronger_layout)

83

With the first set of constraints, the resulting SdP had 31 free variables to optimize, while the second SdP had to deal with 83. Tighter energy bounds usually come at the cost of higher computational costs. 

In [ ]:
assert solver.ojimetro(simple_layout) < solver.ojimetro(stronger_layout)

### Alternative methods to lower bound the ground state energy
We implement two other methods to obtain lower bounds of many-body Hamiltonians. We rewrite the Anderson bound [[2]](https://journals.aps.org/pr/abstract/10.1103/PhysRev.83.1260) and the method introduced by Uskov and Lichovskiy in [[3]](https://iopscience.iop.org/article/10.1088/1742-6596/1163/1/012057) in SdP form to compare with the proposed methodology above. The SdP formulation ensures that the obtained result is the actual global minimum.

The main limitation of these methods is not taking into account compatibility constraints between the reduced density matrices spanning the system. Furthermore, they rely on symmetries (such as translational invariance) and can, thus, not be used to solve inhomogeneous systems. Conversely, the approach we introduced above can be nautrally complemented by introducing additional constratints stemming from any previously known symmetry from the system. 

In [ ]:
#| export
class SdPEnergyAndersonSolver(SdPEnergySolver):
    """Solver to compute bounds to the ground state energy of local Hamiltonians implementing the
    methods described in [2] (see references below). Finds the so-called Anderson bound. Assumes
    the system is one-dimensional."""
    
    def solve(self, interactions, cluster_size=3):
        "Computes the Anderson bound for a one-dimensional system with `interactions`."
        cluster_size, n_clusters = self._get_clusters(interactions, cluster_size)

        sites = np.arange(cluster_size)
        rho = picos.HermitianVariable('rho'+','.join(map(str, sites)),
                                      (2**cluster_size, 2**cluster_size))
        objective = self._build_energy_objective(sites, rho, interactions)
        
        problem = picos.Problem(solver=self.solver)
        problem.add_constraint(rho >> 0)
        problem.add_constraint(picos.trace(rho) == 1)
        problem.set_objective('min', objective)
        
        try:    
            problem.solve()
            result = np.real(objective.value)
        except: 
            print(problem)
            result = 0.
        return n_clusters*result
        
    def ojimetro(self, cluster_size):
        "Estimates the cost involved in finding the Anderson bound for the given cluster size."
        cluster_size = self._get_cluster_size(cluster_size)
        return 2**(2*cluster_size)
    
    def _build_energy_objective(self, sites, rho, interactions):
        objective = 0
        for support, h in interactions: 
            common, _, idx = np.intersect1d(support, sites, return_indices=True)
            if len(common) == len(support): 
                rdm = rho.partial_trace(self._complementary_system(idx, len(sites)))
                objective += (rdm | h) # Tr(rdm·H')
        return objective
        
    def _get_clusters(self, interactions, cluster_size):
        "Find the number of clusters needed for the system."
        system_size = max([s for sites, _ in interactions for s in sites]) + 1
        cluster_size = self._get_cluster_size(cluster_size)
        if system_size % (cluster_size - 1) != 0:
            raise ValueError(f"The number of qubits {system_size} must be divisible by " +
                             "cluster_size - 1.")
        else: 
            n_clusters = system_size // (cluster_size - 1) # 1D nearest-neighbor interaction
        return cluster_size, n_clusters
    
    @staticmethod
    def _get_cluster_size(cluster_size):
        "Checks the `cluster_size` format and handles it accordingly."
        if isinstance(cluster_size, (int, float)): return int(cluster_size)
        elif isinstance(cluster_size, Iterable): return max([len(l) for l in cluster_size])
        else: raise ValueError("`cluster_size` should be an int or a layout.")        

In [ ]:
#| export
class SdPEnergyUskovLichkovskiySolver(SdPEnergyAndersonSolver):
    """Solver to compute ground state energy bounds of local Hamiltonians. It follows the method
    introduced in [3] (see references below) to improve over the Anderson bound. The method assumes
    the reduced states have certain symmetries, such as translational invariance."""
    
    def solve(self, interactions, cluster_size=3):
        "Computes the Uskov and Lichkovskiy bound for a one-dimensional Hamiltonian."
        cluster_size, n_clusters = self._get_clusters(interactions, cluster_size)
        
        sites = np.arange(cluster_size)
        rho = picos.HermitianVariable('rho'+','.join(map(str, sites)), (2**cluster_size, 2**cluster_size))
        objective = self._build_energy_objective(sites, rho, interactions) 
        
        symmetry_constraints = self._get_symmetry_constraints(sites, rho)
        problem = picos.Problem(solver = 'cvxopt')
        problem.add_constraint(rho >> 0)
        problem.add_constraint(picos.trace(rho) == 1)
        problem.add_list_of_constraints(symmetry_constraints)
        problem.set_objective('min', objective)
        
        try:    
            problem.solve()
            result = np.real(objective.value)
        except: 
            print(problem)
            result = 0.
        return n_clusters*result
        
        
    def _get_symmetry_constraints(self, sites, rho):
        # Symmetry constraints: TI and specular symmetry in the cluster
        x = picos.Constant('x', [[0, 1], [1, 0]])
        y = picos.Constant('y', [[0, -1j], [1j, 0]])
        z = picos.Constant('z', [[1, 0], [0, -1]])
        Id = picos.Constant('Id', [[1, 0], [0, 1]])

        def sigma_term(idx): return tensor_prod(idx, x) + tensor_prod(idx, y) + tensor_prod(idx, z)

        def tensor_prod(idx, s):
            "Tensor product of operator `s` acting on indexes `idx`. Fills rest with Id."
            matrices = [s if i in idx else Id for i in sites]
            prod = matrices[0]
            for i in range(1, len(sites)): prod = prod @ matrices[i]
            return prod
        
        constraints = []
        idx = [sites[i:i+2] for i in range(len(sites)-1)]
        for i in range(len(idx)//2):
            left, right = sigma_term(idx[i]), sigma_term(idx[-(i+1)])
            adj = sigma_term(idx[i+1])
            # Specular symmetry
            constraints.append((left | rho) == (right | rho)) # Tr(sigma_01*rho) = Tr(sigma_{n-1,n}*rho)
            # Translational invariance
            constraints.append((left | rho) == (adj | rho)) # Tr(sigma_01*rho) = Tr(sigma_12*rho)
            
        return constraints

In [ ]:
#| include: false
from bounce.hamiltonian import XYZHamiltonian

We can quickly reproduce the results from [[3]](https://iopscience.iop.org/article/10.1088/1742-6596/1163/1/012057) (see Table 2) by properly adjusting the cluster size and using the same Hamiltonian.

In [ ]:
N = 8
chain = Chain1D(N)
B, J = 0, 1
H = XYZHamiltonian(chain, B, J)

In [ ]:
solver_anderson = SdPEnergyAndersonSolver()
solver_uskov = SdPEnergyUskovLichkovskiySolver()

In [ ]:
cluster_size = 5
layout = [np.sort(np.arange(i, i+cluster_size)%N) for i in np.arange(0, N, cluster_size-3)]

In [ ]:
#| slow
solver_anderson.solve(H.to_sdp(), cluster_size)/N, solver_uskov.solve(H.to_sdp(), cluster_size)/N, solver.solve(H.to_sdp(), layout)/N

(-1.9278862525095595, -1.8685170845449277, -1.868517086761974)

## Entanglement witnessing

We take an energy-based approach to build entanglement witnesses here. The main principle is, given a Hamiltonian, we compute the minimum possible energy obtainable within the set of separable states. Then, if we find a state that yields a lower energy, it means that it is entangled [[4]](https://journals.aps.org/pra/abstract/10.1103/PhysRevA.71.010301). 

Formally, $$\Delta E = \langle H\rangle - E_{\text{sep}}$$ denotes the energy gap between the expected energy of a state, $\langle H\rangle = \text{Tr}\left[\rho H\right]$, and the minimum energy obtainable by separable states, $E_{\text{sep}}$. If $\Delta E < 0$, $\rho$ is an entangled state, as it is beyond the set of separable states. The larger the difference, $|\Delta E|$, the more robust the entanglement. Actually, $|\Delta E|$ quantifies the energy the system must receive to become separable.

In [ ]:
#| export
class SdPWitnessSolver(SdPEnergySolver):
    """Solver to detect entanglement from energy bounds. It follows the formalism described in [1]
    to obtain a lower bound of the minimum energy for separable states. If we can find a state with
    a lower energy than that, the state is entangled."""
    
    def solve(self, interactions, layout):
        """Creates and solves the SdP associated to the given layout and Hamiltonian interactions.
        The result is a lower bound of the minimum energy for separable states."""
        layout = simplify_layout(layout)
        problem = picos.Problem(solver=self.solver)
        variables = [(site, picos.HermitianVariable('rho'+','.join(map(str, site)), 
                                                    (2**len(site), 2**len(site)))) for site in layout]
        objective = self._build_energy_objective(variables, interactions)
        
        compatibility_constraints = self._get_compatibility_constraints(variables)
        ppt_constraints = self._get_ppt_constraints(variables)
        problem.add_list_of_constraints([rho >> 0 for _, rho in variables])
        problem.add_list_of_constraints([picos.trace(rho) == 1 for _, rho in variables])
        problem.add_list_of_constraints(compatibility_constraints)
        problem.add_list_of_constraints(ppt_constraints)
        problem.set_objective('min', objective)
        
        try:    
            problem.solve()
            result = np.real(objective.value)
        except: 
            result = None
        return result
    
    @staticmethod
    def _get_ppt_constraints(variables):
        """Returns the constraints to be imposed such that the reduced density matrices are
        positive under partial transposition (PPT) to approximate the set of separable states."""
        constraints = []
        for sites, rho in variables:
            idx = np.arange(len(sites))
            for size in range(1, len(sites)):
                constraints.append([picos.partial_transpose(rho, 
                                                            subsystems=map(int, subsystem)) >> 0 
                                    for subsystem in combinations(idx, size)])
        return [c for rdm_constraints in constraints for c in rdm_constraints]

The `SdPWitnessSolver` provides a lower bound for $E_{\text{sep}}$ given a `Hamiltonian`. We build upon the `SdPEnergySolver` imposing additional constraints to enforce separability on our states. However, imposing such conditions to their full extent is extremely costly. Instead, we impose that our states are positive under partial transposition (PPT), which is a relaxation that is exact in the one-dimensional limit. Hence, the bound on $E_{\text{sep}}$. 

Let's see an example! First of all, we need to build our problem.

In [ ]:
N = 6
chain = Chain1D(N)
B, J = 1, 2
H = XXHamiltonian(chain, B, J)

Now we can instantiate our solver.

In [ ]:
solver_witness = SdPWitnessSolver()

We define a layout of 2-body terms.

In [ ]:
layout = fill_layout([np.sort([i, (i+1)%N]) for i in range(N)], N)
layout

[array([0, 1]),
 array([1, 2]),
 array([2, 3]),
 array([3, 4]),
 array([4, 5]),
 array([0, 5])]

And now we're ready to solve it.

In [ ]:
solver_witness.solve(H.to_sdp(), layout)

-12.749999991956518

In [[4]](https://journals.aps.org/pra/abstract/10.1103/PhysRevA.71.010301) we find analytic expressions of $E_{\text{sep}}$ for some Hamiltonians, including the XX model we consider here.  

In [ ]:
def exact_XX_sep_bound(N, B, J, d):
    b = B/J/d
    if b <= 2:
        return -d*N*J*(1+b**2/4)
    return -d*N*J*b

In [ ]:
exact_XX_sep_bound(N, B, J, 1)

-12.75

We see that, since we've chosen to implement the Hamiltonian in a one-dimensional chain, the result we obtained is exact. Now we can use the `SdPEnergySolver` to obtain the ground state energy and compare it to the separable threshold.

In [ ]:
solver_energy = SdPEnergySolver()
solver_energy.solve(H.to_sdp(), layout)

-23.99999993375356

The ground state is entangled! :)

If we now change the Hamiltonian parameters, we can land on a separable ground state.

In [ ]:
B, J = 5, 1
H = XXHamiltonian(chain, B, J)

In [ ]:
solver_witness.solve(H.to_sdp(), layout), solver_energy.solve(H.to_sdp(), layout)

(-29.9999999937129, -29.999999800727085)

In this case, both energies match. This confirms that the ground state is, indeed, a separable state. 

# References
[1] B. Requena, G. Muñoz-Gil, M. Lewenstein, V. Dunjko, J. Tura. [Certificates of quantum many-body properties assisted by machine learning](https://arxiv.org/abs/2103.03830). *arXiv:2103.03830 (2021)*

[2] P. W. Anderson. [Limits on the Energy of the Antiferromagnetic Ground State](https://journals.aps.org/pr/abstract/10.1103/PhysRev.83.1260). *Physical Review* **83**, 1260 (1951)

[3] F. Uskov and O. Lychkovskiy. [A variational lower bound on the ground state of a many-body system and the squaring parametrization of density matrices](https://iopscience.iop.org/article/10.1088/1742-6596/1163/1/012057). *Journal of Physics: Conference Series* **1163** 012057 (2019)

[4] G. Tóth. [Entanglement Witnesses in Spin Models](https://journals.aps.org/pra/abstract/10.1103/PhysRevA.71.010301). *Physical Review A* **71** 010301 (2005).

#| hide
# Export-

In [ ]:
#| include: false
from nbdev import nbdev_export
nbdev_export()

Converted 00_environment.ipynb.
Converted 01_agents.ipynb.
Converted 02_budget_profiles.ipynb.
Converted 03_hamiltonian.ipynb.
Converted 04_training.ipynb.
Converted 05_utils.ipynb.
Converted 06_sdp.ipynb.
Converted index.ipynb.
